# Лабораторная №3. Обработка последовательностей
## Цели работы:
1. Реализовать алгоритм LSTM.
2. Сравнить решение с марковской цепью (автоматом).
3. Анализ результатов.
## Набор данных
Выберите любой доступный достаточно длинный текст. Разбейте текст по предложениям на строки. Максимально сократите число различных символов в тексте: приведите все буквы к нижнему регистру, уберите лишние знаки препинания. Если используемая модель будет плохо или медленно обучаться, разбейте текст на строки по словам. В таком случае в качестве набора данных можно взять какой-либо словарь.
Используя one-hot кодирование, получите векторное представление каждого символа. В том числе и символа конца строки (точки или пробела), после которого генерация строки прекращается. Хотя её также следует искусственно ограничить максимальной длинной строки.
## Задание
Обучите модель на выбранном тексте. И используйте обученную модель для генерации продолжения строки по её началу. Задачу стоит решить двумя моделями: LSTM и марковской цепью (автоматом).
## LSTM
Обучите LSTM предсказывать следующий символ по предыдущим. Обучать стоит по одной строке. Для обучения можно использовать любую подходящую функцию ошибки (например MSE или CrossEntropy). Для обучения используйте адаптивный градиентный спуск. Для предсказания следующего символа можно использовать arg-max вектора ответа либо использовать случайный символ с вероятностью, полученной soft-arg-max-ом.
## Марковская цепь
Выберите число последних символов n, которые будут учитываться для генерации нового символа. Не путайте это число с числом символов, из которых будет генерироваться продолжение строки, они могут различаться!
Обучите марковскую цепь — автомат, в котором состояние — это суффикс текущей строки длины n, а переходы — очередной генерируемый символ. Каждый переход снабжён некоторым числовым значением — вероятностью перехода, которую и нужно оценить на выбранном тексте.
Для предсказания следующего символа можно использовать arg-max вектора вероятностей перехода из текущего состояния либо использовать случайный символ с соответствующей вероятностью.

In [1]:
import re

import keras
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, LSTM
from collections import defaultdict

# Набор данных.
### Возьмём первые 10 глав произведения "Война и Мир", преобразуем текст. В тексте примерно 11500 слов.
1. Открываем файл "war_and_peace.txt" с кодировкой "utf-8" и считываем содержимое файла в строку text.
2. Приводим все буквы строки text к нижнему регистру.
3. Заменяем все знаки вопроса и восклицания на точки с помощью регулярного выражения.
4. Заменяем все символы кроме кириллических символов, пробелов и точек на пробелы с помощью регулярного выражения.
5. Заменяем все повторяющиеся пробелы на один пробел.
6. Выводим на экран содержимое строки text и количество пробелов в ней.


In [3]:
text = open('war_and_peace.txt', encoding="utf-8").read().lower()

text = re.sub(r'[!?]', '.', text)
text = re.sub(r'[^а-я\s.]', ' ', text)
text = re.sub(r'\s+', ' ', text)

print(text)
print(text.count(' '))

ну что князь генуя и лукка стали не больше как поместья поместья фамилии буонапарте. нет я вам вперед говорю если вы мне не скажете что у нас война если вы позволите себе защищать все гадости все ужасы этого антихриста право я верю что он антихрист я вас больше не знаю вы уже не друг мой вы уже не мой верный раб как вы говорите. ну здравствуйте здравствуйте. я вижу что я вас пугаю садитесь и рассказывайте. так говорила в июле года известная анна павловна шерер фрейлина и приближенная императрицы марии федоровны встречая важного и чиновного князя василия первым приехавшего на ее вечер. анна павловна кашляла несколько дней у нее был грипп как она говорила грипп был тогда новое слово употреблявшееся только редкими а потому она не дежурила и не выходила из дому. в записочках разосланных утром с красным лакеем было написано без различия во всех если у вас граф или князь нет в виду ничего лучшего и если перспектива вечера у бедной больной не слишком вас пугает то я буду очень рада видеть вас

In [5]:
text_chars = sorted(list(set(text)))
chars_index_map = {char: ind for ind, char in enumerate(text_chars)}
indexes_char_map = {ind: char for ind, char in enumerate(text_chars)}

print(chars_index_map)
print(indexes_char_map)

{' ': 0, '.': 1, 'а': 2, 'б': 3, 'в': 4, 'г': 5, 'д': 6, 'е': 7, 'ж': 8, 'з': 9, 'и': 10, 'й': 11, 'к': 12, 'л': 13, 'м': 14, 'н': 15, 'о': 16, 'п': 17, 'р': 18, 'с': 19, 'т': 20, 'у': 21, 'ф': 22, 'х': 23, 'ц': 24, 'ч': 25, 'ш': 26, 'щ': 27, 'ъ': 28, 'ы': 29, 'ь': 30, 'э': 31, 'ю': 32, 'я': 33}
{0: ' ', 1: '.', 2: 'а', 3: 'б', 4: 'в', 5: 'г', 6: 'д', 7: 'е', 8: 'ж', 9: 'з', 10: 'и', 11: 'й', 12: 'к', 13: 'л', 14: 'м', 15: 'н', 16: 'о', 17: 'п', 18: 'р', 19: 'с', 20: 'т', 21: 'у', 22: 'ф', 23: 'х', 24: 'ц', 25: 'ч', 26: 'ш', 27: 'щ', 28: 'ъ', 29: 'ы', 30: 'ь', 31: 'э', 32: 'ю', 33: 'я'}


Выбираем число символов, на которые смотрим, чтобы определить следующий.
dataX - массив со всеми подстроками текста длины n.
dataY - массив с символами, которые идут сразу после каждой подстроки

In [6]:
n = 10

dataX, dataY = [], []

for sentence in text.split('.'):
    dataX += [sentence[i:i + n] for i in range(len(sentence) - n)]
    dataY += [sentence[i + n] for i in range(len(sentence) - n)]

print(dataX[:15])
print(dataY[:15])

['ну что кня', 'у что княз', ' что князь', 'что князь ', 'то князь г', 'о князь ге', ' князь ген', 'князь гену', 'нязь генуя', 'язь генуя ', 'зь генуя и', 'ь генуя и ', ' генуя и л', 'генуя и лу', 'енуя и лук']
['з', 'ь', ' ', 'г', 'е', 'н', 'у', 'я', ' ', 'и', ' ', 'л', 'у', 'к', 'к']


### Сделаем one-hot кодирование и получим векторное представление каждого символа и подстроки.
X - массив, в котором по номеру строки и символа хранится его векторное представление (то есть массив с векторными представлениями подстрок)
Y - массив с векторным представление следующего символа для каждой подстроки.

In [6]:
amount_sentences = len(dataX)

X = np.zeros((amount_sentences, n, len(chars_index_map)))
Y = np.zeros((amount_sentences, len(chars_index_map)))

for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, chars_index_map[char]] = 1
    Y[i, chars_index_map[dataY[i]]] = 1

print(X)
print(Y)
print(X.shape)
print(Y.shape)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[[0. 0. 0.

## LSTM.

### Создадим модель и обучим её.
Этот код обучает модель для рекуррентной нейронной сети (LSTM) на вводах (X) и соответствующих целевых значениях (Y) с оптимизатором Adam и функцией потерь категориальной кросс-энтропии. Он также использует обратные вызовы (callbacks) в Keras, в частности ModelCheckpoint, чтобы сохранять лучшие веса модели во время обучения. Модель будет обучаться на протяжении 40 эпох.

In [40]:
checkpoint = ModelCheckpoint("weights/weight.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model = keras.Sequential([LSTM(256, input_shape=(X.shape[1], X.shape[2])),Dense(Y.shape[1], activation='softmax')])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(X, Y, epochs=40, callbacks=callbacks_list)

Epoch 1/40
1996/1996 [==============================] - ETA: 0s - loss: 2.5478
Epoch 1: loss improved from inf to 2.54775, saving model to weights\improvement-01-2.5478-bigger.hdf5
1996/1996 [==============================] - 52s 25ms/step - loss: 2.5478
Epoch 2/40
1995/1996 [============================>.] - ETA: 0s - loss: 2.0922
Epoch 2: loss improved from 2.54775 to 2.09210, saving model to weights\improvement-02-2.0921-bigger.hdf5
1996/1996 [==============================] - 51s 26ms/step - loss: 2.0921
Epoch 3/40
1994/1996 [============================>.] - ETA: 0s - loss: 1.8664
Epoch 3: loss improved from 2.09210 to 1.86649, saving model to weights\improvement-03-1.8665-bigger.hdf5
1996/1996 [==============================] - 51s 26ms/step - loss: 1.8665
Epoch 4/40
1994/1996 [============================>.] - ETA: 0s - loss: 1.7104
Epoch 4: loss improved from 1.86649 to 1.71048, saving model to weights\improvement-04-1.7105-bigger.hdf5
1996/1996 [==============================]

### Предскажем несколько предложений. Будем продолжать случайные подстроки текста.

Этот код генерирует случайные последовательности символов (предложения) на основании модели обучения, которая была ранее построена на основе текстовых данных. Код проходит по циклу пять раз и для каждой итерации выбирается случайный индекс из набора данных. Затем, для выбранного индекса, определяется текущий шаблон предложения, который становится отправной точкой для последующего предсказания следующего символа и генерации нового предложения.

In [72]:
for _ in range(5):
    random_index = np.random.randint(amount_sentences - 1)
    current_pattern = dataX[random_index]
    print(f'Начало: {current_pattern}')
    ans = current_pattern

    for _ in range(60):
        x = np.zeros((1, n, len(text_chars)))

        for i, char in enumerate(current_pattern):
            x[0, i, chars_index_map[char]] = 1

        predict = model.predict(x, verbose=0)
        next_symbol = indexes_char_map[np.argmax(predict)]
        current_pattern = current_pattern[1:] + next_symbol
        ans += next_symbol

        if next_symbol == '.':
            break

    print(f'Предсказывание: {ans}')
    print()

Начало:  и приехал
Предсказывание:  и приехала теперь чтобы выхлопотать определение в гвардию вот вам моя

Начало: талии свои
Предсказывание: талии свои маленькие было были и могостой и отчиянные я знать петербур

Начало: авало чтоб
Предсказывание: авало чтобы вся эта компания совсем мне опротивела из ерудал кабаяь у 

Начало: у и чудном
Предсказывание: у и чудному государю и он прямо будет переведен в гвардию вот вам моя 

Начало: нно против
Предсказывание: нно противоположного мнения и оглядывая слушателей и оживление улыбки 



## Марковская цепь.

### Построим марковскую цепь.

В первом цикле кода инициализируется словарь `mark_chain` с помощью `defaultdict`, где ключом является фраза-реплика, а значением еще один `defaultdict`, где ключами являются последующие реплики и значение - количество раз, когда эта последующая реплика последовала после данной фразы-реплики.

Затем для каждой фразы-реплики считается сумма всех значений во вложенном словаре и затем каждое значение делится на эту сумму, чтобы получить вероятности каждой последующей реплики при данной фразе-реплике.

In [78]:
mark_chain = defaultdict(lambda: defaultdict(lambda: 0))
for sentence, continuation in zip(dataX, dataY):
    mark_chain[sentence][continuation] += 1

for con in dataX:
    s = sum([v for v in mark_chain[con].values()])

    for v in mark_chain[con]:
        mark_chain[con][v] /= s

### Предскажем несколько предложений с помощью марковской цепи.

Этот код генерирует последовательности символов на основе цепей Маркова. Он выбирает случайный шаблон из набора данных (dataX), затем генерирует последовательность символов, используя цепь Маркова, которая строится на основе символов в шаблоне и их связей с символами, которые следуют за ними. Результатом является предсказание последовательности символов, которые могут появиться после выбранного начального шаблона

In [75]:
for _ in range(5):
    random_index = np.random.randint(amount_sentences - 1)
    current_pattern = dataX[random_index]

    print(f'Начало: {current_pattern}')
    ans = current_pattern

    for _ in range(60):
        next_symbols_probabilities = mark_chain[current_pattern]
        p = [v for v in next_symbols_probabilities.values()]

        if len(p) == 0:
            ans += '.'
            break

        next_symbol = list(next_symbols_probabilities.items())[np.random.choice(len(p), p=np.array(p))]
        current_pattern = current_pattern[1:] + next_symbol[0]
        ans += next_symbol[0]

    print(f'Предсказывание: {ans}')
    print()

Начало: менили ска
Предсказывание: менили сказал князь андрей но в этом месте рассказал как мадемуазель ж

Начало:  до того к
Предсказывание:  до того какое сочинение хорошо или дурно а особенно нынче любезен.

Начало:  что несмо
Предсказывание:  что несмотря на все его недостатки он невольно все обратились на его 

Начало: бающееся л
Предсказывание: бающееся лицо пьера улыбнулся и улыбкой открыл неправильные черные зуб

Начало: два лакея 
Предсказывание: два лакея за карета делать визиты .

